## 完全ではないが，ある程度データは取れる
- 現状は日本基準で５つまでなら確認が取れた（年度末二件，第一四半期，第二四半期それぞれ一件ずつ）
## 問題点
- 企業ごとにタグの使い方が若干異なる（予期しないエラーがまだまだ発生しそう）
- 米国基準のものが一つだけ取得できたが，型式がひどい




In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import math
import csv

#選択行削除
import re
from xml.sax.saxutils import unescape

In [2]:
def normalize_XBRL(XBRL):
    with open(XBRL, encoding='utf-8') as f:
        html = f.read()
    # 「（」「）」はすべて半角に
    html = html.replace("（", "(")
    html = html.replace("）", ")")
    #tableタグだけ残す，それ以外のタグで囲まれていない部分を削除
    html  = re.sub("<table", "table\n<", html)
    html  = re.sub("</table", "tableend\n<", html)
    html  = re.sub("<br />", "", html)
    html  = re.sub("<td", "td<", html)
    html  = re.sub("</td>", "tdend\n", html)
    html  = re.sub("</tr>", "trend", html)
    
    text=""
    for i in html:
        text=text+i
        if i ==">":
            text=re.sub("<.*>", "", text)
            text=re.sub("{.*}", "", text)
            if text[-1:]!="\n":
                text = text+"\n"
    text1=text

    #テキスト処理
    text1=re.sub("&#160;", "None", text1)
    text1=re.sub("△\n", "-", text1)
    text1=re.sub("△", "-", text1)
    text1=re.sub(".root", "-", text1)
    text1=re.sub(",", "", text1)
    text1=re.sub("．", ".", text1)
    text1=re.sub("\ufeff\n", "", text1)
    text1=re.sub("－", "None", text1)
    text1=re.sub("：", "", text1)
    text1=re.sub(" ", "", text1)
    text1=re.sub("\n\n", "\n", text1)
    text1=re.sub("　", "", text1)
    text1=re.sub("０", "0", text1)
    text1=re.sub("１", "1", text1)
    text1=re.sub("２", "2", text1)
    text1=re.sub("３", "3", text1)
    text1=re.sub("４", "4", text1)
    text1=re.sub("５", "5", text1)
    text1=re.sub("６", "6", text1)
    text1=re.sub("７", "7", text1)
    text1=re.sub("８", "8", text1)
    text1=re.sub("９", "9", text1)
    

    return(text1)

In [3]:
def text_tag(text):
    #予測がいつまでなのかを取得
    pre_cnt=0
    pre = ""
    for i in range(len(text),-1,-1):
        if "連結業績予想(" in text[i-7:i]:
            pre_cnt=i
            break
    for i in range(pre_cnt,-1,-1):
        if "." in text[i-1:i]:
            pre = text[i:pre_cnt-1]
            break

    #tdで囲まれた部分の改行は削除して，tdを削除
    tx=""
    text1=""
    td_flag=False
    for i in text:
        #print(i)
        if "tdend" in text1:
            td_flag=False
        if "td" in text1:
            td_flag=True

        if td_flag:
            if i=="\n":
                i=""
        text1+=i
    text1=re.sub("tdend", "", text1)
    text1=re.sub("td", "\n", text1)
    text1=re.sub(r"\n+", "\n", text1)
    return(text1,pre)

In [54]:
def create_text_list(text):
    
    #tableタグで囲まれた部分をリストに入れる
    #tableタグで囲まれた部分をリストに入れる
    text_list = []
    table_cnt_first=0
    table_cnt_end=0
    non_text_list = []
    non_table_cnt_first=0
    append_text=""
    non_text=""
    for i in range(1,len(text)):
        if text[i-8:i] == "tableend":
        #tableタグに囲まれた文章をlistに追加
            table_cnt_end = i-8
            append_text=text[table_cnt_first:table_cnt_end]+"\n"
            append_text=re.sub("trend\ntrend","trend\n",append_text)
            append_text=re.sub("tableendtable","",append_text)
            append_text=re.sub(r'\n+', "\n",append_text)
            text_list.append(append_text)
            table_cnt_first=0
            table_cnt_end=0
            #tableタグに囲まれている部分を追加したので囲まれていない部分のカウント初期化
            non_table_cnt_first=i

    #tableタグの始まりの判定    
        elif text[i-5:i] == "table":
            if table_cnt_first==0:
                table_cnt_first=i

            #tableタグに囲まれていない部分をlistに追加
            non_text=text[non_table_cnt_first:i-5]
            non_text_list.append(non_text)
    
    #listの中に（注）がある部分は削除する         
    pop_list=[]
    cnt=0
    for i in text_list:

        #if "(注)" in i :
          #  pop_list.append(cnt)
        if "(参考)" in i:
            pop_list.append(cnt)
        elif "(役職名)" in i:
            pop_list.append(cnt)
        cnt +=1

    for i in pop_list[::-1]:
        #print(i)
        text_list.pop(i)
    return(text_list,non_text_list)

In [5]:
def create_table_list(text_list):
    #\nが１０個以下の部分は表ではないので削除
    table_list = []
    kari_list = []
    pop_cnt = 0
    end_flag=False
    for i in text_list:
        if i.count("\n") > 9:
            table_list.append(i)
    #発行株式数の表だけ作りが違うため今回は削除
    table_list = table_list[:8]

    #配当金のグラフは変則的なので最後に回す
    for cnt, i in enumerate(table_list):
        if "配当金" in i:
            text = i
            if "配当金総額" in text:
                end_flag=True
            text= text.replace("(","")
            text = text.replace(")","")
            text = re.sub("連結trend","連結",text)
            kari_list=text
            pop_cnt = cnt

        if "通期" in i:
            table_list[cnt]= table_list[cnt].replace("売上高","check売上高")
            break
    table_list.pop(pop_cnt)
    table_list.append(kari_list)
    #tableが出てきたら削除
    #最初のtrendまでに\nが一回しか出ていなければ削除
    return(table_list,end_flag)

In [6]:
def create_list(table_list):
    number_list = []
    col_list=[]
    number = []
    col=""
    num_cnt=0
    col_cnt=0
    len_col=0
    add_col=0
    for table in table_list:
        if "table" in table:
            table=table[table.find("table")+5:]
        for i in range(0,len(table)):
            if table[i:i+5] !="trend":
                if table[i:i+1] == "\n":
                    col=table[col_cnt+1:i]
                    add_col+=1
                    if col=="" and col_list!=[] and len_col==0:
                        len_col=len(col_list)
                    if col!="":
                        col_list.append(col)
                    col_cnt=i
            else:
                col=table[col_cnt+1:i]
                col_list.append(col)
                #追加したコラムが１つだけなら削除してやり直し
                if add_col==1:
                    add_col=0
                    col_list.pop(-1)
                    col_cnt=i+5
                    continue
                add_col=0
                num_cnt=i+5
                break   
        #column以降にある数字の部分を回す
        for i in range(num_cnt,len(table)+1):
            if table[i:i+1] == "\n":
                if table[i-5:i]=="trend":
                    if number!=[]:
                        number.append(table[num_cnt+1:i-5])
                        number_list.append(number)
                        number=[]
                        num_cnt=i

                elif table[i-5:i]!="trend":
                    number.append(table[num_cnt+1:i])
                    num_cnt=i
    
    [col_list.insert(k, col_list[k]) for k in range(len_col-1,-1,-1)]
    
    return(col_list,number_list)

In [7]:
def create_None(column,number):
    #Noneが連続している場合は削除
    drop_list=[]
    number_list=[]
    for i in range(0,len(column)):
        if column[i] =="None":
            #if column[i+1]=="None":
            drop_list.append(i)
    for i in drop_list[::-1]:
        #print(i)
        column.pop(i)
        
    #numberにNoneを入れる
    for num in number:
        if "円銭" in num or "百万円" in num:
            if "None"  not in num:
                num.insert(0,"None")
        number_list.append(num)
    
    return(column,number)

In [8]:
def create_DataFrame(number,end_flag):
    new_df = pd.DataFrame
    new_df2 = pd.DataFrame
    new_df3 = pd.DataFrame
    new_df4=pd.DataFrame
    kari_df = pd.DataFrame
    index_list = ["None"]
    df4_flag=False
    #学期末なら空リスト，それ以外なら，頭に追加
    if end_flag:
        index2_list=[]
    else:
        index2_list=["None"]
        
    marge_cnt = 0
    number_cnt = 1
    cnt = 0
    none_cnt=0
    flag=False
    flag_data=False
    #indexが同じ部分は横につなげる
    for i in range(1,len(number[number_cnt])):
        number_cnt +=1
        if number[i][0]== "None":
            new_df = pd.DataFrame([number[k][1:] for k in range(0,3)],index=[index_list])
            marge_cnt = i
            break
        else:
            index_list.append(number[i][0])
            cnt=i+1
    #同じインデックスでは横に並べる
    len_num = len(index_list)
    k = 1
    for i in range(cnt,len(number)):
        if number[i][0]==index_list[k]:
            k+=1
            if k ==len_num:
                kari_df = pd.DataFrame([number[marge_cnt+j][1:] for j in range(0,i-marge_cnt+1)],index=[index_list])
                new_df=pd.concat([new_df,kari_df],axis=1)
                marge_cnt = i+1
                k=1
                flag=True
        elif number[i][0] =="None":
            none_cnt=i
        elif number[i][0]!="None" and flag:
            [index2_list.append(number[k][0]) for k in range(i-1,len(number))]
            cnt=i
            break
     #それ以外を別のDataFrameに   
    for i in range(none_cnt,len(number)-1):
        if number[i+1][0]=="None":
            new_df2 = pd.DataFrame([number[none_cnt+k][1:] for k in range(0,i-none_cnt+1)],index=[index2_list[0:i-none_cnt+1]])
            none_cnt=i+1
            break

    #dataftame３
    if end_flag:
        #dataftame３
        for i in range(none_cnt,len(number)-1):
            if number[i+1][0]=="None":
                new_df3 = pd.DataFrame([number[none_cnt+k][1:] for k in range(0,i-none_cnt+1)],
                                       index=index2_list[new_df2.shape[0]:new_df2.shape[0]+i-none_cnt+1])
                none_cnt=i
                flag_data = True
                break

        new_df3 = pd.DataFrame([number[none_cnt+k][1:] for k in range(0,i-none_cnt+2)],
                               index=index2_list[new_df2.shape[0]:new_df2.shape[0]+i-none_cnt+2])
        
    else:
        for i in range(none_cnt,len(number)-1):
            if number[i+1][0]=="None":
                new_df3 = pd.DataFrame([number[none_cnt+k][1:] for k in range(0,i-none_cnt+1)],
                                       index=index2_list[new_df2.shape[0]:new_df2.shape[0]+i-none_cnt+1])
                none_cnt=i
                flag_data = True
                break

        if flag==False:
            #print(number[i:],none_cnt,i)
            #print(index2_list[new_df2.shape[0]:new_df2.shape[0]+i-none_cnt+1])
            new_df3 = pd.DataFrame([number[none_cnt+k][1:] for k in range(0,i-none_cnt+2)],
                                   index=index2_list[new_df2.shape[0]-1:new_df2.shape[0]+i-none_cnt+1])

        if none_cnt != len(number):
            new_df4=pd.DataFrame([number[none_cnt+k][1:] for k in range(1,len(number)-none_cnt)],
                                 index=[index2_list[-(len(number)-none_cnt-1):]])
            df4_flag=True

    return(new_df,new_df2,new_df3,new_df4,df4_flag)

In [9]:
def create_only_DataFrame(number,end_flag):
    new_df = pd.DataFrame
    kari_df = pd.DataFrame
    index_list = ["None"]
    df4_flag=False
    #学期末なら空リスト，それ以外なら，頭に追加
    if end_flag:
        index2_list=[]
    else:
        index2_list=["None"]
        
    marge_cnt = 0
    number_cnt = 1
    cnt = 0
    none_cnt=0
    flag=False
    flag_data=False
    #indexが同じ部分は横につなげる
    for i in range(1,len(number[number_cnt])):
        number_cnt +=1
        if number[i][0]== "None":
            new_df = pd.DataFrame([number[k][1:] for k in range(0,3)],index=[index_list])
            marge_cnt = i
            break
        else:
            index_list.append(number[i][0])
            cnt=i+1
    #同じインデックスでは横に並べる
    len_num = len(index_list)
    k = 1
    for i in range(cnt,len(number)):
        if number[i][0]==index_list[k]:
            k+=1
            if k ==len_num:
                kari_df = pd.DataFrame([number[marge_cnt+j][1:] for j in range(0,i-marge_cnt+1)],index=[index_list])
                new_df=pd.concat([new_df,kari_df],axis=1)
                marge_cnt = i+1
                k=1
                flag=True
        elif number[i][0] =="None":
            none_cnt=i
        elif number[i][0]!="None" and flag:
            [index2_list.append(number[k][0]) for k in range(i-1,len(number))]
            cnt=i
            break

    return(new_df)

In [10]:
def create_column(df1,df2,df3,df4,column,end_flag,df4_flag):
    sh1=df1.shape
    sh2 = df2.shape
    sh3=df3.shape
    sh4=df4.shape
    column2=[]
    column3=[]
    column4=[]
    col_cnt=0
    #column1をリセット
    df1.columns = range(sh1[1])
    df1.columns=[column[i] + df1[i][0] for i in range(0,sh1[1])]

    if "check" in column[sh1[1]]:
        column[sh1[1]] = re.sub("check","",column[sh1[1]])
        column2=column[sh1[1]:sh1[1]+int(sh2[1]/2)+1]
        [column2.insert(k, column2[k]) for k in range(len(column2)-2,-1,-1)]
        df2.columns=[column2[i] + df2[i][0] for i in range(0,sh2[1])]

        #column3の作成
        column3=column[sh1[1]+int(sh2[1]/2)+1:]
        if end_flag:
            for cnt,i in enumerate(column3):
                if "四半期末" in i:
                    col_cnt=cnt
                    break
            column3=column[sh1[1]+int(sh2[1]/2)+1+col_cnt:]
            for i in range(1,cnt+1):
                column3.append(column[sh1[1]+int(sh2[1]/2)+1+i])
        #column3をリセット
        df3.columns = range(sh3[1])
        df3.columns=[column3[i] + df3[i][0] for i in range(0,sh3[1])]
    
    else:
        #column2の作成
        column2=column[sh1[1]:sh1[1]+sh2[1]]
        #column2をリセット
        df2.columns = range(sh2[1])
        df2.columns=[column2[i] + df2[i][0] for i in range(0,sh2[1])]

        column[sh1[1]+sh2[1]] = re.sub("check","",column[sh1[1]+sh2[1]])
        column3=column[sh1[1]+sh2[1]:sh1[1]+sh2[1]+int(sh3[1]/2)+1]
        [column3.insert(k, column3[k]) for k in range(len(column3)-2,-1,-1)]
        df3.columns=[column3[i] + df3[i][0] for i in range(0,sh3[1])]
        
        #column4
        if df4_flag:
            #print(column[-sh4[1]:])
            column4 = column[-sh4[1]:]
            df4.columns = range(sh4[1])
            df4.columns=[column4[i] + df4[i][0] for i in range(0,sh4[1])]
    
    return(df1,df2,df3,df4)

In [11]:
def create_df_column(df1,df):
    sh1=df.shape
    columns_df=[]
    column_df1=[ i for i in df1.columns]
    columns_df=[column[i] for i in range(0,len(df.columns))]
    df.columns = range(sh1[1])
    df.columns=columns_df
    return(df)

In [12]:
def create_only_column(df1,column,end_flag):
    sh1=df1.shape
    col_cnt=0
    #column1をリセット
    df1.columns = range(sh1[1])
    df1.columns=[column[i] + df1[i][0] for i in range(0,sh1[1])]
    return(df1)

In [13]:
def select_df(df2,df3,df4,pre_term):
    
    try:
        if "通期" in df2.index or "予想" in df2.index:
            return(df2)
        elif "通期" in df3.index or "予想" in df3.index:
            return(df3)
        elif "通期" in df4.index or "予想" in df4.index:
            return(df4)
        
        else:
            index_list=[]
            index_list = [i[0] for i in df2.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df2)
                pass
            
            index_list = [i[0] for i in df3.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df3)
                pass
            
            index_list = [i[0] for i in df4.index]
            for j in index_list:
                if pre_term[:4] in j:
                    return(df4)
                pass
    except:
        pass

In [14]:
def select_df(df2,df3,df4,pre_term):
    
    try:
        if "通期" in df2.index or "予想" in df2.index:
            return(df2)
        elif "通期" in df3.index or "予想" in df3.index:
            return(df3)
        elif "通期" in df4.index or "予想" in df4.index:
            return(df4)
        
    except:
        pass

In [15]:
#textの取得，連結業績予想文
for pre_text in text_list:
    if "業績予想" in pre_text:
        print(pre_text)

NameError: name 'text_list' is not defined

In [16]:
#token保存
import pickle
out='xbrl_df/df2_none_list.pkl'
with open(out, 'rb') as p:
    non_list=pickle.load(p)

In [351]:
pre_com_list[69]

92

In [368]:
i=0
non_list[i]

'xbrl_data/16050/20190808485390.htm'

In [369]:
text=normalize_XBRL(non_list[i])
text,pre_term = text_tag(text)
text_list=create_text_list(text)
table_list,end_flag=create_table_list(text_list)
column,number=create_list(table_list)
column,number = create_None(column,number)
df1,df2,df3,df4,flag=create_DataFrame(number,end_flag)
df=select_df(df2,df3,df4,pre_term)
df=create_df_column(df1,df)

In [370]:
df

,売上高,売上高,営業利益,営業利益,経常利益,経常利益,親会社株主に帰属する四半期純利益,親会社株主に帰属する四半期純利益,1株当たり四半期純利益
None,百万円,％,百万円,％,百万円,％,百万円,％,円銭
第2四半期(累計),581000,32.6,276000,21.9,260000,5.3,49000,44.0,33.56
通期,988000,None,472000,None,460000,None,90000,None,61.64
②期末自己株式数,2019年12月期1Ｑ,2123234,株,2019年3月期,2123800,株,None,None,None
③期中平均株式数(四半期累計),2019年12月期1Ｑ,1460199941,株,2019年3月期1Ｑ,1460357100,株,None,None,None


In [293]:
df2

,0,1,2,3
None,百万円,百万円,％,円銭
2020年3月期,367365,157986,42.3,1808.65
2019年3月期,390524,159227,40.1,1819.74


In [371]:
out_csv2

'err_data2/92200/20220513545645_2.csv'

In [372]:
df2_none_list=[]
import tqdm
pre_text_list=[]
pre_com_list=[]
for i in tqdm.tqdm(range(0,len(non_list))):
    out_csv1='err_data2/{}/{}_1.csv'.format(non_list[i][10:15],non_list[i][16:-4])
    out_csv2='err_data2/{}/{}_2.csv'.format(non_list[i][10:15],non_list[i][16:-4])
    out_txt2='err_data2/{}/{}_err.txt'.format(non_list[i][10:15],non_list[i][16:-4])
    text=normalize_XBRL(non_list[i])
    text,pre_term = text_tag(text)
    text_list=create_text_list(text)
    table_list,end_flag=create_table_list(text_list)
    column,number=create_list(table_list)
    column,number = create_None(column,number)
    try:
        df1,df2,df3,df4,flag=create_DataFrame(number,end_flag)
        df=select_df(df2,df3,df4,pre_term)
        df=create_df_column(df1,df)
        df.to_csv(out_csv2)
    except:
        pre_com_list.append(i)
        pre_text_li=""
        #textの取得，連結業績予想文
        for pre_text in text_list:
            if "業績予想" in pre_text:
                #print(pre_text)
                pre_text_li+=pre_text
        pre_text_list.append(pre_text_li)
        #print("****************************************************")

100%|███████████████████████████████████████| 1154/1154 [00:33<00:00, 34.82it/s]


In [373]:
len(pre_text_list)

676

In [375]:
pre_com_list

[1,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 52,
 53,
 54,
 55,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 113,
 114,
 117,
 118,
 119,
 120,
 133,
 134,
 138,
 139,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 195,
 196,
 197,
 198,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 214,
 216,
 217,
 235,
 238,
 240,
 241,
 242,
 243,
 246,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274

In [378]:
dic={}
for i in range(0,len(pre_com_list)):
     dic[non_list[pre_com_list[i]][10:15]]=1

In [387]:
dic_li=[i for i in (dic.keys())]

In [381]:
import pickle
out='stopword/not_com_list.pkl'
with open(out, 'rb') as p:
    not_com_list=pickle.load(p)

In [383]:
int(not_com_list[0])

57130

In [391]:
for i in dic_li:
    for j in not_com_list:
        if i==str(int(j)):
            print(i)
            continue

22160
28010
49180
54010
69180
89250
97170
36220
31330
61750
35410
70630


In [393]:
len(dic_li)-12

169

In [251]:
for cnt,i in enumerate(pre_text_list):
    if i =="":
        print(cnt)

4
5
10
16
17
23
24
25
27
28
29
37
39
40
41
42
43
44
45
69
70
71
72
73
74
75
76
77
78
85
92
93
111
112
113
114
115
116
117
118
120
121
122
124
126
132
133
134
135
136
137
138
139
142
144
145
156
157
158
159
160
161
162
163
164
165
166
167
168
188
189
190
191
192
193
194
195
196
197
198
200
201
202
203
210
212
213
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
234
267
268
269
270
271
272
273
274
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
344
355
356
357
359
360
361
363
364
365
367
368
369
370
371
372
373
375
376
377
379
380
381
384
385
386
387
388
390
391
393
394
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
426
429
430
431
432
434
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
491
494
495
496
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
533
541
542
543
546
548
550
551
552
553
554
555
556
558
559
576
584
585
586
587
588
608
623
634
635
637
638
63

In [341]:
pre_com_list[40]

60

In [24]:
import tqdm
li=[]
for i in tqdm.tqdm(range(0,len(non_list))):
    text=normalize_XBRL(non_list[i])
    text,pre_term = text_tag(text)
    text_list=create_text_list(text)
    table_list,end_flag=create_table_list(text_list)
    li.append(len(table_list))
    #column,number=create_list(table_list)
    #column,number = create_None(column,number)

100%|███████████████████████████████████████| 1509/1509 [00:39<00:00, 37.84it/s]


In [25]:
for cnt,i in enumerate(li):
    if i>6:
        break

In [27]:
cnt

1508

In [28]:
len(li)

1509

In [37]:
text=normalize_XBRL(non_list[4])
text,pre_term = text_tag(text)
text_list=create_text_list(text)
table_list,end_flag=create_table_list(text_list)

In [38]:
table_list

['\n売上高\n営業利益\n経常利益\n親会社株主に帰属する四半期純利益trend\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\n2022年3月期第1四半期\n1641\n―\n-1038\n―\n-953\n―\n-958\n―trend\n2021年3月期第1四半期\n3241\n-40.0\n-846\n―\n-831\n―\n-1481\n―trend\n',
 '\n1株当たり四半期純利益\n潜在株式調整後1株当たり四半期純利益trend\n円銭\n円銭trend\n2022年3月期第1四半期\n-109.12\n―trend\n2021年3月期第1四半期\n-168.71\n―trend\n',
 '\n総資産\n純資産\n自己資本比率trend\n百万円\n百万円\n％trend\n2022年3月期第1四半期\n52651\n9699\n18.4trend\n2021年3月期\n50297\n10853\n21.5trend\n',
 '\n新規\n―\n社\n(社名)\n、\n除外\n―\n社\n(社名)\ntrend\n',
 '\n①期末発行済株式数(自己株式を含む)\n2022年3月期1Q\n8808778株\n2021年3月期\n8808778株trend\n②期末自己株式数\n2022年3月期1Q\n25383株\n2021年3月期\n25266株trend\n③期中平均株式数(四半期累計)\n2022年3月期1Q\n8783455株\n2021年3月期1Q\n8783835株trend\n',
 '\n2.配当の状況table\n年間配当金trend\n第1四半期末\n第2四半期末\n第3四半期末\n期末\n合計trend\n円銭\n円銭\n円銭\n円銭\n円銭trend\n2021年3月期\n―\n0.00\n―\n0.00\n0.00trend\n2022年3月期\n―\ntrend\n2022年3月期予想\n―\n―\n―\n―trend\n']

In [42]:
text

'第1四半期決算短信〔日本基準〕(連結)\n.xbrl_bodyMargin\n.xbrl_bodyMargin\n.xbrl_tse\ntable\n2022年3月期第1四半期決算短信〔日本基準〕(連結)2021年8月12日table\n上場会社名\n常磐興産株式会社\n上場取引所\n東trendtableendtable\nコード番号\n96750\nURL\nhttp://www.joban-kosan.comtrendtableendtable\n代表者\n(役職名)\n代表取締役社長\n(氏名)\n西澤順一trend\n問合せ先責任者\n(役職名)\n取締役執行役員管理担当\n(氏名)\n下山田敏博\nTEL\n0246-43-0569trendtableendtable\n四半期報告書提出予定日\n2021年8月12日\ntrendtableendtable\n配当支払開始予定日\n―\ntrendtableendtable\n四半期決算補足説明資料作成の有無\n無\ntrend\n四半期決算説明会開催の有無\n無\ntrendtableend(百万円未満切捨て)trendtableendtable\n1.2022年3月期第1四半期の連結業績(2021年4月1日～2021年6月30日)table\n(1)連結経営成績(累計)\n(％表示は、対前年同四半期増減率)trendtableendtable\n売上高\n営業利益\n経常利益\n親会社株主に帰属する四半期純利益trend\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\ntrend\n2022年3月期第1四半期\n1641\n―\n-1038\n―\n-953\n―\n-958\n―trend\n2021年3月期第1四半期\n3241\n-40.0\n-846\n―\n-831\n―\n-1481\n―trendtableendtable\n(注)包括利益\n2022年3月期第1四半期-1081百万円\n(―％)\n2021年3月期第1四半期-1281百万円\n(―％)trendtableendtable\n1株当たり四半期純利益\n潜在株式調整後1株当たり四半期純利益trend\n円銭\n円銭trend\ntrend\n2022年3月期第1四半期\n-109.12\n―t

True

In [92]:
ind = pre_term[:4]
ind in index_list

False

In [83]:
pre_term[:4] in df2.index[1][0]

True

In [89]:
index_list = [i[0] for i in df2.index]
index_list

['None', '2022年３月期第１四半期', '2021年３月期']

In [95]:
ind in index_list[1]

True

In [98]:
df

,資産合計百万円,資本合計百万円,親会社の所有者に帰属する持分百万円,親会社所有者帰属持分比率％
None,百万円,百万円,百万円,％
2022年３月期第１四半期,1944521,1284100,1165628,59.9
2021年３月期,1885999,1222983,1113923,59.1


In [69]:
df4

,株円銭,2021年３月期円銭,290814015円銭,株円銭,年間配当金円銭
None,円銭,円銭,円銭,円銭,円銭
2021年３月期,None,22.00,None,99.00,121.00
2022年３月期,None,None,None,None,None
2022年３月期予想,None,71.00,None,62.00,133.00


In [70]:
df2

,資産合計百万円,資本合計百万円,親会社の所有者に帰属する持分百万円,親会社所有者帰属持分比率％
None,百万円,百万円,百万円,％
2022年３月期第１四半期,1944521,1284100,1165628,59.9
2021年３月期,1885999,1222983,1113923,59.1


In [15]:
df1.columns

Index(['売上高百万円', '売上高％', '営業利益百万円', '営業利益％', '経常利益百万円', '経常利益％',
       '親会社株主に帰属する四半期純利益百万円', '親会社株主に帰属する四半期純利益％', '1株当たり四半期純利益円銭',
       '潜在株式調整後1株当たり四半期純利益円銭'],
      dtype='object')

In [19]:
for i,col in enumerate(df1.columns):
    print(i)
    print(col)

0
売上高百万円
1
売上高％
2
営業利益百万円
3
営業利益％
4
経常利益百万円
5
経常利益％
6
親会社株主に帰属する四半期純利益百万円
7
親会社株主に帰属する四半期純利益％
8
1株当たり四半期純利益円銭
9
潜在株式調整後1株当たり四半期純利益円銭


In [74]:
"通　期" in df2.index

True

In [47]:
df4[0]

TypeError: 'type' object is not subscriptable

## 以下notebook

In [57]:
for i in range(0,len(non_list)):
    if non_list[i][10:15]=="85930":
        print(i)
        break

658


In [59]:
non_list[659]

'xbrl_data/85930/20211111432521.htm'

In [60]:
with open(non_list[659], encoding='utf-8') as f:
    html = f.read()
html  = re.sub(">", ">\n\n", html)
print(html)

﻿<?xml version="1.0" encoding="utf-8"?>


<html version="-//XBRL International//DTD XHTML Inline XBRL 1.0//EN" xmlns="http://www.w3.org/1999/xhtml" xmlns:tse-qcedjpsm-85930="http://www.xbrl.tdnet.info/jp/tse/tdnet/qc/edjp/sm/85930/20211112385930" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:tse-ed-t="http://www.xbrl.tdnet.info/taxonomy/jp/tse/tdnet/ed/t/2014-01-12" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:ix="http://www.xbrl.org/2008/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2011-07-31" xmlns:num="http://www.xbrl.org/dtr/type/numeric" xmlns:nonnum="http://www.xbrl.org/dtr/type/non-numeric">

<head>

<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />

<meta http-equiv="X-UA-Compatible" conte

In [76]:
#non_list

In [80]:
df = pd.read_csv('xbrl_df/13010/20170619410034_1.csv',encoding="utf-8")

In [81]:
df

,Unnamed: 0,売上高百万円,売上高％,営業利益百万円,営業利益％,経常利益百万円,経常利益％,親会社株主に帰属する四半期純利益百万円,親会社株主に帰属する四半期純利益％,1株当たり四半期純利益円銭,潜在株式調整後1株当たり四半期純利益円銭
0,None,百万円,％,百万円,％,百万円,％,百万円,％,円銭,円銭
1,30年3月期第1四半期,56844,8.9,979,109.6,1103,190.1,754,36.8,71.80,66.56
2,29年3月期第1四半期,52206,4.1,467,70.8,380,-22.7,551,74.0,52.47,48.37


In [82]:
text=normalize_XBRL(non_list[659])
text,pre_term = text_tag(text)
text_list,non_text_list=create_text_list(text)
table_list,end_flag=create_table_list(text_list)
column,number=create_list(table_list)
column,number = create_None(column,number)
df1,df2,df3,df4,flag=create_DataFrame(number,end_flag)
df=select_df(df2,df3,df4,pre_term)
df=create_df_column(df1,df)

In [85]:
df

,売上高,売上高,営業利益,営業利益,経常利益,経常利益
None,百万円,％,円銭,None,None,None
通期,95000,71.7,66.17,None,None,None
②None期末自己株式数,2022年3月期2Ｑ,31232500,株,2021年3月期,4368016,株
③None期中平均株式数(四半期累計),2022年3月期2Ｑ,1435598327,株,2021年3月期2Ｑ,891125958,株
第1四半期末,第2四半期末,第3四半期末,期末,合計,None,None


In [51]:
#tableタグで囲まれた部分をリストに入れる
text_list = []
table_cnt_first=0
table_cnt_end=0
non_text_list = []
non_table_cnt_first=0
append_text=""
non_text=""
for i in range(1,len(text)):
    if text[i-8:i] == "tableend":
        #tableタグに囲まれた文章をlistに追加
        table_cnt_end = i-8
        append_text=text[table_cnt_first:table_cnt_end]+"\n"
        append_text=re.sub("trend\ntrend","trend\n",append_text)
        append_text=re.sub("tableendtable","",append_text)
        append_text=re.sub(r'\n+', "\n",append_text)
        text_list.append(append_text)
        table_cnt_first=0
        table_cnt_end=0
        #tableタグに囲まれている部分を追加したので囲まれていない部分のカウント初期化
        non_table_cnt_first=i
        
    #tableタグの始まりの判定    
    elif text[i-5:i] == "table":
        if table_cnt_first==0:
            table_cnt_first=i
        
        #tableタグに囲まれていない部分をlistに追加
        non_text=text[non_table_cnt_first:i-5]
        non_text_list.append(non_text)

In [88]:
column

['売上高',
 '売上高',
 '営業利益',
 '営業利益',
 '経常利益',
 '経常利益',
 '親会社株主に帰属する四半期純利益',
 '親会社株主に帰属する四半期純利益',
 '(注)1.包括利益',
 '2022年3月期第2四半期',
 '74310',
 '百万円',
 '(517.5％)',
 '2021年3月期第2四半期',
 '12034',
 '百万円',
 '(None％)',
 '1株当たり四半期純利益',
 '潜在株式調整後1株当たり四半期純利益',
 '総資産',
 '純資産',
 '自己資本比率',
 '親会社株主に帰属する当期純利益',
 '1株当たり当期純利益',
 '①None期末発行済株式数(自己株式を含む)',
 '2022年3月期2Ｑ',
 '1466912244',
 '株',
 '2021年3月期',
 '895834160',
 '株',
 '年間配当金']

In [127]:
column_dic={
    "売上高":"Sales",
    "営業利益":"Operating",
    "事業利益":"Business",
    "経常利益":"Ordinary",
    "親会社株主に帰属する当期純利益":"ParentNetIncome",
    "親会社株主に帰属する四半期純利益":"QuarterParentNetIncome",
    "1株当たり当期純利益":"OneNetIncome",
    "1株当たり四半期純利益":"QuarterOneNetIncome"
}

In [109]:
li=[i for i in column_dic.keys()]
for cnt,i in enumerate(column):
    if i in li:
        column[cnt]=column_dic[i]

In [110]:
column

['Sales',
 'Sales',
 'Operating',
 'Operating',
 'Ordinary',
 'Ordinary',
 'QuarterParentNetIncome',
 'QuarterParentNetIncome',
 '(注)1.包括利益',
 '2022年3月期第2四半期',
 '74310',
 '百万円',
 '(517.5％)',
 '2021年3月期第2四半期',
 '12034',
 '百万円',
 '(None％)',
 '1株当たり四半期純利益',
 '潜在株式調整後1株当たり四半期純利益',
 '総資産',
 '純資産',
 '自己資本比率',
 'ParentNetIncome',
 'OneNetIncome',
 '①None期末発行済株式数(自己株式を含む)',
 '2022年3月期2Ｑ',
 '1466912244',
 '株',
 '2021年3月期',
 '895834160',
 '株',
 '年間配当金']

In [ ]:
def create_list(table_list):
    number_list = []
    col_list=[]
    number = []
    col=""
    num_cnt=0
    col_cnt=0
    len_col=0
    add_col=0
    for table in table_list:
        if "table" in table:
            table=table[table.find("table")+5:]
        for i in range(0,len(table)):
            if table[i:i+5] !="trend":
                if table[i:i+1] == "\n":
                    col=table[col_cnt+1:i]
                    add_col+=1
                    if col=="" and col_list!=[] and len_col==0:
                        len_col=len(col_list)
                    if col!="":
                        col_list.append(col)
                    col_cnt=i
            else:
                col=table[col_cnt+1:i]
                col_list.append(col)
                #追加したコラムが１つだけなら削除してやり直し
                if add_col==1:
                    add_col=0
                    col_list.pop(-1)
                    col_cnt=i+5
                    continue
                add_col=0
                num_cnt=i+5
                break   
        #column以降にある数字の部分を回す
        for i in range(num_cnt,len(table)+1):
            if table[i:i+1] == "\n":
                if table[i-5:i]=="trend":
                    if number!=[]:
                        number.append(table[num_cnt+1:i-5])
                        number_list.append(number)
                        number=[]
                        num_cnt=i

                elif table[i-5:i]!="trend":
                    number.append(table[num_cnt+1:i])
                    num_cnt=i
    
    [col_list.insert(k, col_list[k]) for k in range(len_col-1,-1,-1)]
    
    return(col_list,number_list)

In [111]:
text=normalize_XBRL(non_list[659])
text,pre_term = text_tag(text)
text_list,non_text_list=create_text_list(text)
table_list,end_flag=create_table_list(text_list)
column,number=create_list(table_list)

In [114]:
text_list

['\nNonetable\ntrend\n',
 '\n',
 '\n(1)連結経営成績(累計)\n(％表示は対前年同期増減率)trend\n',
 '\nNone\n売上高\n営業利益\n経常利益\n親会社株主に帰属する四半期純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\n2022年3月期第2四半期\n833358\n83.3\n58151\n80.6\n59089\n75.7\n60482\n146.0trend\n2021年3月期第2四半期\n454604\nNone\n32199\nNone\n33632\nNone\n24586\nNonetrend\n',
 '\n(注)1.包括利益\n2022年3月期第2四半期\n74310\n百万円\n(517.5％)\nNone\n2021年3月期第2四半期\n12034\n百万円\n(None％)trend\n2.当社(旧会社名三菱ＵＦＪリース株式会社)は、2021年4月1日付で日立キャピタル株式会社と経営統合を行っており、2021年3月期第2四半期の連結経営成績においては、旧三菱ＵＦＪリース株式会社の実績を記載しております。3.会計方針の変更にともない、2021年3月期第2四半期の連結経営成績においては、遡及修正を行っております。そのため、2021年3月期第2四半期の対前年同期増減率は記載しておりません。trend\n',
 '\nNone\n1株当たり四半期純利益\n潜在株式調整後1株当たり四半期純利益trend\nNone\n円銭\n円銭trend\n2022年3月期第2四半期\n42.13\n42.03trend\n2021年3月期第2四半期\n27.59\n27.49trend\n',
 '\nNone\n総資産\n純資産\n自己資本比率trend\nNone\n百万円\n百万円\n％trend\n2022年3月期第2四半期\n9459998\n1252420\n13.0trend\n2021年3月期\n6014896\n817906\n13.4trend\n',
 '\nNone\n年間配当金trend\n第1四半期末\n第2四半期末\n第3四半期末\n期末\n合計trend\nNone\n円銭\n円銭\n円銭\n円銭\n円銭trend\n2

In [128]:
li=[]
n_cnt=0
col=""
column_li=[i for i in column_dic.keys()]
#一番可能性が高い表をlistのトップに持ってくる
for cnt,table in enumerate(text_list):
    n_cnt=0
    col=""
    for i,word in enumerate(table,1):
        if word=="\n":
            #columnになる可能性のある単語がcolumn辞書に存在する場合は高い確率で目的の表であると考えられる
            col = table[n_cnt:i-1]
            if col in column_li:
                li.append(table)
                break
            n_cnt=i

In [129]:
li

['\nNone\n売上高\n営業利益\n経常利益\n親会社株主に帰属する四半期純利益trend\nNone\n百万円\n％\n百万円\n％\n百万円\n％\n百万円\n％trend\n2022年3月期第2四半期\n833358\n83.3\n58151\n80.6\n59089\n75.7\n60482\n146.0trend\n2021年3月期第2四半期\n454604\nNone\n32199\nNone\n33632\nNone\n24586\nNonetrend\n',
 '\nNone\n1株当たり四半期純利益\n潜在株式調整後1株当たり四半期純利益trend\nNone\n円銭\n円銭trend\n2022年3月期第2四半期\n42.13\n42.03trend\n2021年3月期第2四半期\n27.59\n27.49trend\n',
 '\nNone\n親会社株主に帰属する当期純利益\n1株当たり当期純利益trend\nNone\n百万円\n％\n円銭trend\n通期\n95000\n71.7\n66.17trend\n']